# LCMS Viewer Intro Notebook
* Based on https://github.com/google/earthengine-community/blob/master/datasets/scripts/LCMS_Visualization.js
Copyright 2025 The Google Earth Engine Community Authors
 	
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
 	
* Example script for visualizing LCMS change summaries, land cover, and land use.
* A more in-depth visualization of LCMS products is available at: [https://apps.fs.usda.gov/lcms-viewer/index.html](https://apps.fs.usda.gov/lcms-viewer/index.html)      
* Contact sm.fs.lcms@usda.gov with any questions or specific data requests.



[![github](https://img.shields.io/badge/-see%20sources-white?logo=github&labelColor=555)](https://github.com/gee-community/geeviz/blob/master/examples/lcmsViewerExampleNotebook.ipynb) 
[![github](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gee-community/geeViz/blob/master/examples/lcmsViewerExampleNotebook.ipynb)

In [1]:
#Boiler plate
#Import modules
import os,sys
sys.path.append(os.getcwd())


try:
    from  geeViz.geeView import *
except:
    !python -m pip install geeViz
    from  geeViz.geeView import *


print('Done')

Initializing GEE
Successfully initialized
Done


## First, we'll take a look at the foundation of the `geeViz.geeView` module - the LCMS Data Explorer

* This viewer framework serves as the foundation for this module

In [ ]:
#This notebook breaks down various pieces of LCMS data visualization available in this interactive viewer
IFrame(src='https://apps.fs.usda.gov/lcms-viewer/', width='100%', height='500px')

## Create your own LCMS Viewer

* You can create your own instance of the LCMS Data Explorer using `geeViz.geeView`

In [2]:
#Clear any layers added to Map object
#If map is not cleared, layers are simply appended to the existing list of layers if layers have been added previously
Map.clearMap()

#############################################################################
### Define visualization parameters ###
#############################################################################
startYear = 1985
endYear = 2023
lossYearPalette = ['ffffe5', 'fff7bc', 'fee391', 'fec44f', 'fe9929',
                   'ec7014', 'cc4c02']
gainYearPalette = ['c5ee93', '00a398']
durationPalette = ['BD1600', 'E2F400','0C2780']

lossYearViz = {'min': startYear, 'max': endYear, 'palette': lossYearPalette};
gainYearViz = {'min': startYear, 'max': endYear, 'palette': gainYearPalette};
durationViz = {'min': 1, 'max': 5, 'palette': durationPalette};

#############################################################################
### Define functions ###
#############################################################################

#Convert given code to year that number was present in the image.
def getMostRecentChange(c, code):
	def wrapper(img):
		yr = ee.Date(img.get('system:time_start')).get('year')
		return ee.Image(yr).int16().rename(['year']).updateMask(img.eq(code)).copyProperties(img,['system:time_start'])
	return c.map(wrapper)

In [7]:
#############################################################################
### Bring in LCMS annual outputs ###
#############################################################################

lcms = ee.ImageCollection('USFS/GTAC/LCMS/v2023-9')
props = lcms.first().toDictionary().getInfo()
print('Properties:',props)
bandNames =  lcms.first().bandNames().getInfo()
print('Available study areas:', lcms.aggregate_histogram('study_area').keys().getInfo())
print('Available LCMS products',bandNames)
print('Learn more about visualization of LCMS products here', 'https://apps.fs.usda.gov/lcms-viewer/')

#Filter out study area
# lcms = lcms.filter(ee.Filter.eq('study_area','CONUS'))

#Set up time periods to compare land cover and land use
earlySpan = [startYear, startYear+4]
lateSpan = [endYear-4, endYear]


Properties: {'Change_class_names': ['Stable', 'Slow Loss', 'Fast Loss', 'Gain', 'Non-Processing Area Mask'], 'Change_class_palette': ['3d4551', 'f39268', 'd54309', '00a398', '1b1716'], 'Change_class_values': [1, 2, 3, 4, 5], 'Land_Cover_class_names': ['Trees', 'Tall Shrubs & Trees Mix (SEAK Only)', 'Shrubs & Trees Mix', 'Grass/Forb/Herb & Trees Mix', 'Barren & Trees Mix', 'Tall Shrubs (SEAK Only)', 'Shrubs', 'Grass/Forb/Herb & Shrubs Mix', 'Barren & Shrubs Mix', 'Grass/Forb/Herb', 'Barren & Grass/Forb/Herb Mix', 'Barren or Impervious', 'Snow or Ice', 'Water', 'Non-Processing Area Mask'], 'Land_Cover_class_palette': ['005e00', '008000', '00cc00', 'b3ff1a', '99ff99', 'b30088', 'e68a00', 'ffad33', 'ffe0b3', 'ffff00', 'aa7700', 'd3bf9b', 'ffffff', '4780f3', '1b1716'], 'Land_Cover_class_values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'Land_Use_class_names': ['Agriculture', 'Developed', 'Forest', 'Non-Forest Wetland', 'Other', 'Rangeland or Pasture', 'Non-Processing Area Mask']

---
* Raw LCMS outputs are available for more customized analysis
* Double click anywhere within CONUS to plot the time series of raw LCMS outputs

In [ ]:
#############################################################################
### Add full raw model outputs ###
#############################################################################
Map.clearMap()
#Separate products
raw_change = lcms.select(['Change_Raw_.*'])
raw_land_cover = lcms.select(['Land_Cover_Raw_.*'])
raw_land_use = lcms.select(['Land_Use_Raw_.*'])

#Shorten names
raw_change_bns = [bn for bn in bandNames if bn.find('Change_Raw_') > -1]
raw_land_cover_bns = [bn for bn in bandNames if bn.find('Land_Cover_Raw_') > -1]
raw_land_use_bns = [bn for bn in bandNames if bn.find('Land_Use_Raw_') > -1]

raw_change_bns_short = [i.split('_Probability_')[-1] for i in raw_change_bns]
raw_land_cover_bns_short = [i.split('_Probability_')[-1] for i in raw_land_cover_bns]
raw_land_use_bns_short = [i.split('_Probability_')[-1] for i in raw_land_use_bns]

raw_change = raw_change.select(raw_change_bns,raw_change_bns_short)
raw_land_cover = raw_land_cover.select(raw_land_cover_bns,raw_land_cover_bns_short)
raw_land_use = raw_land_use.select(raw_land_use_bns,raw_land_use_bns_short)


#Add to map
Map.addLayer(raw_land_use,{'reducer':ee.Reducer.max(),'min':0,'max':30,'opacity':1,'addToLegend':False,'queryParams':{'palette':props['Land_Use_class_palette'],'yLabel':'Model Confidence'}},'Raw LCMS Land Use Model Probability',True) 
Map.addLayer(raw_land_cover,{'reducer':ee.Reducer.max(),'min':0,'max':30,'opacity':1,'addToLegend':False,'queryParams':{'palette':props['Land_Cover_class_palette'],'yLabel':'Model Confidence'}},'Raw LCMS Land Cover Model Probability',True) 
Map.addLayer(raw_change,{'reducer':ee.Reducer.max(),'min':0,'max':30,'bands':'Fast_Loss,Gain,Slow_Loss','opacity':1,'addToLegend':False,'queryParams':{'palette':props['Change_class_palette'],'yLabel':'Model Confidence'}},'Raw LCMS Change Model Probability',True) 


Map.centerObject(lcms.first())
Map.turnOnInspector()
Map.setQueryDateFormat('YYYY')
Map.view()


Adding layer: Raw LCMS Land Use Model Probability
Adding layer: Raw LCMS Land Cover Model Probability
Adding layer: Raw LCMS Change Model Probability
Setting default query date format to: YYYY
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:8001/geeView/ already serving.
cwd a:\GEE\gee_py_modules_package\geeViz\examples
geeView URL: http://localhost:8001/geeView/?projectID=nlcd-tcc&accessToken=ya29.a0AeXRPp5TKi07CQNRZpseyKqj6NsM6KY4rWMepm9qtM_ZXJbzH06oCdbfspH8j8Tq6Ll4XxIPmnm5g2PlAQIZo7cf5seHrNCA-mV0UF0_v5BZH7-lUt_0QtkybK-FJus5R-YqKfelDbl4j7STPyx9RtPswmToKQN1rZcTBeMhzf8aCgYKAX4SARESFQHGX2MiAKhF5RJjuqS9HVCKinUCyQ0178&accessTokenCreationTime=1742581911067


In [9]:
Map.clearMap()
#############################################################################
### Visualize Land Use change ###
#############################################################################
lu = lcms.select(['Land_Use'])
earlyLu = lu.filter(ee.Filter.calendarRange(earlySpan[0], earlySpan[1], 'year'))
lateLu = lu.filter(ee.Filter.calendarRange(lateSpan[0], lateSpan[1], 'year'))
Map.addLayer(earlyLu, {'reducer':ee.Reducer.mode(),'autoViz':True}, 'Early Land Use Mode ({}-{})'.format(earlySpan[0],earlySpan[1]), True)
Map.addLayer(lateLu, {'reducer':ee.Reducer.mode(),'autoViz':True,'opacity':0}, 'Recent Land Use Mode ({}-{})'.format(lateSpan[0],lateSpan[1]), True);



#############################################################################
### Visualize Land Cover change ###
#############################################################################
lc = lcms.select(['Land_Cover'])
earlyLc = lc.filter(ee.Filter.calendarRange(earlySpan[0], earlySpan[1], 'year'))
lateLc = lc.filter(ee.Filter.calendarRange(lateSpan[0], lateSpan[1], 'year'))
Map.addLayer(earlyLc, {'reducer':ee.Reducer.mode(),'autoViz':True}, 'Early Land Cover Mode ({}-{})'.format(earlySpan[0],earlySpan[1]), True);
Map.addLayer(lateLc, {'reducer':ee.Reducer.mode(),'autoViz':True,'opacity':0}, 'Recent Land Cover Mode ({}-{})'.format(lateSpan[0],lateSpan[1]), True);


Map.turnOnInspector()
Map.setQueryDateFormat('YYYY')
Map.view()

Adding layer: Early Land Use Mode (1985-1989)
Adding layer: Recent Land Use Mode (2018-2022)
Adding layer: Early Land Cover Mode (1985-1989)
Adding layer: Recent Land Cover Mode (2018-2022)
Setting default query date format to: YYYY
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:8001/geeView/ already serving.
cwd a:\GEE\gee_py_modules_package\geeViz\examples
geeView URL: http://localhost:8001/geeView/?projectID=lcms-292214&accessToken=ya29.a0AcM612zDgRjgEmDi_yYVNIvaVY2azV8cVYE6ETwGhWAsQQ1CzIwHreik7o0TClPAyf9Okm6xURUPlDUyxkodC15EDFza8rYZUEk61FhReaxwzrdMOUL5_HU6aO0OOvHpQxlWjqmBS9rcU8_iLAB27qv9vvosyhFo8fyHpjDDcXgaCgYKAb8SARESFQHGX2MionRzqxWQQ-3VjFP8htnWOA0178


In [10]:
Map.clearMap()
#############################################################################
### Visualize Change products ###
#############################################################################

#Select the change band. Land_Cover and Land_Use are also available.
change = lcms.select(['Change'])

#Convert to year collection for a given code.
slowLossYears = getMostRecentChange(change, 2)
fastLossYears = getMostRecentChange(change, 3)
gainYears = getMostRecentChange(change, 4)

#Find the most recent year.
mostRecentSlowLossYear = slowLossYears.max()
mostRecentFastLossYear = fastLossYears.max()
mostRecentGainYear = gainYears.max()

#Find the duration.
slowLossDuration = slowLossYears.count()
fastLossDuration = fastLossYears.count()
gainDuration = gainYears.count()

#Add year summaries to the map.
Map.addLayer(mostRecentSlowLossYear, lossYearViz, 'Most Recent Slow Loss Year', True)
Map.addLayer(mostRecentFastLossYear, lossYearViz, 'Most Recent Fast Loss Year', True)
Map.addLayer(mostRecentGainYear, gainYearViz, 'Most Recent Gain Year', True)

#Add durations to the map.
Map.addLayer(slowLossDuration,durationViz, 'Slow Loss Duration', False);
Map.addLayer(fastLossDuration,durationViz, 'Fast Loss Duration', False);
Map.addLayer(gainDuration,durationViz, 'Gain Duration', False);


Map.turnOnInspector()
Map.view()

Adding layer: Most Recent Slow Loss Year
Adding layer: Most Recent Fast Loss Year
Adding layer: Most Recent Gain Year
Adding layer: Slow Loss Duration
Adding layer: Fast Loss Duration
Adding layer: Gain Duration
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:8001/geeView/ already serving.
cwd a:\GEE\gee_py_modules_package\geeViz\examples
geeView URL: http://localhost:8001/geeView/?projectID=lcms-292214&accessToken=ya29.a0AcM612xoftpqheIuHbi9bcWDlmIZ5O9-c9ZwgSBhcAeoIFQL5OuPmgdeReJmy6bCQ9byq4mgyDibRB-EAyImkDoZ4WhcailRq4XoMqD21af80vhmzxv4mBKOlmOhs1ShH5yfgXR-C36i97o5a6gejqxD8Ft2nCxob-x7w9j7y3AaCgYKAXQSARESFQHGX2Midz2BZdheheSq91hgy_2OMQ0178


## Time lapses enable quick visualization of multi-temporal `imageCollections`

In [11]:
# Since any collection can be viewed as a time lapse, let's bring in the change outputs as a timelapse
# These can take time to load since each frame is an individual tile map service
Map.clearMap()
justChange = change.map(lambda img:img.updateMask(ee.Image(img.gt(1).And(img.lt(5))).copyProperties(lcms.first())))
Map.addTimeLapse(justChange,{'autoViz':True},'LCMS Change Time Lapse',False)

Map.setQueryDateFormat('YYYY')
Map.turnOnInspector()
Map.view()

Adding layer: LCMS Change Time Lapse
Setting default query date format to: YYYY
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:8001/geeView/ already serving.
cwd a:\GEE\gee_py_modules_package\geeViz\examples
geeView URL: http://localhost:8001/geeView/?projectID=lcms-292214&accessToken=ya29.a0AcM612wkLZ5nkrRefktkInWr4bG1xoFijDRm4zKqEkOSY5SlJkshjY8c8xPH0opFivgsSvpi-zs-EMn4cghA5Q9vvm7dszHkaSY2xuMWT94ACWv_8wy0D-nhVOi29Oo5fQ5wmHrYHpnRzBiH6SF3RyS-9rr7i3VwptGM3yc1aokaCgYKAUQSARESFQHGX2MiEKkp7i2adXQeQErg-zELAw0178
